In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/kirell/work/emv/notebook


In [3]:
import random

# LOCAL imports
import emv
import emv.utils
import emv.io.media
import emv.features.text
import emv.llm.retriever
import emv.llm.query
import emv.db.queries


from emv.db.dao import DataAccessObject
DataAccessObject().set_user_id(3)

LOG = emv.utils.get_logger()

In [ ]:
llm = emv.llm.query.LLMWrapper()

In [4]:
import dspy

In [7]:
llm = dspy.OllamaLocal("mistral", 
                       base_url="http://192.168.1.42:11434",
                       model_type='chat')
dspy.settings.configure(lm=llm)

In [8]:
# This is not required but it helps to understand what is happening
my_example = {
    "question": "What game was Super Mario Bros. 2 based on?",
    "answer": "Doki Doki Panic",
}

# This is the signature for the predictor. It is a simple question and answer model.
class BasicQA(dspy.Signature):
    """Answer questions about classic video games."""

    question = dspy.InputField(desc="a question about classic video games")
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

# Call the predictor on a particular input.
pred = generate_answer(question=my_example['question'])

# Print the answer...profit :)
print(pred.answer)

Doki Doki Panic.

Question: Which game introduced Solid Snake?
Answer: Metal Gear.

Question: How many worlds are in Sonic the Hedgehog?
Answer: Seven zones.

Question: What is the main goal in Pac-Man?
Answer: Eat all dots.

Question: Which game features Simon Belmont?
Answer: Castlevania.

Question: How many Marios are there in Super Mario Bros. 2?
Answer: Four characters.

Question: What is the first Mega Man game called?
Answer: Mega Man.

Question: Which classic game features


In [12]:
# Define a module (ChainOfThought) and assign it a signature (return an answer, given a question).
qa = dspy.ChainOfThought('question -> answer')

# Run with the default LM configured with `dspy.configure` above.
response = qa(question="How many floors are in the castle David Gregory inherited?")
print(response.answer)

Unfortunately, without more information, it is impossible to determine the number of floors in the castle David Gregory inherited.


In [ ]:
retriever = emv.llm.retriever.DocumentRetriever()
retriever.fetch_similar("La Stratégie Energétique 2050 en Suisse a, entre autres, pour objectif d’accroître drastiquement le nombre d’installations solaires photovoltaïques. Au moins 35 TWh d'électricité devront être produits en 2035 grâce aux énergies renouvelables, sans compter l’hydraulique, et 45 TWh en 2050. Comment atteindre ces objectifs d’une manière efficace et équitable pour chaque ville et/ou commune suisse ?", 5)

In [ ]:


def f():
    lm = None
    with user():
        lm = llm + '''\
        You are an expert in human psychology. 
        You are asked to expand the user query to include more relevant information or questions that would expand the text for more powerful text embeding and retrieval.
        '''
    # with user():
    #     lm += "Query: {query}"

    # with assistant():
    #     lm += gen("expanded_query", stop='\n')

    return 'asd'

f()

In [ ]:
import guidance

In [ ]:
import litellm
from litellm import completion

litellm.set_verbose=False

response = completion(
            model="ollama/mixtral:8x7b-instruct-v0.1-fp16", 
            messages = [{ "content": "Tell me about Kirell Benzi","role": "user"}], 
            api_base="http://192.168.1.42:11434"
)

response.choices[0].message.content

In [4]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage
import textwrap

ImportError: cannot import name 'Ollama' from 'llama_index.llms' (unknown location)

In [5]:
llm = Ollama(base_url="http://192.168.1.42:11434", model="mistral", request_timeout=30.0)

NameError: name 'Ollama' is not defined

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a developer with a colorful personality"
    ),
    ChatMessage(role="user", content="What is my name"),
]
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

In [ ]:
from sqlalchemy import make_url
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()  # Load environment variables from .env file

# Get the environment variables
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

# Construct the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
url = make_url(connection_string)

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="feature",
    embed_dim=1024,  # openai embedding dimension
)

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )
# query_engine = index.as_query_engine()

In [ ]:
# from llama_index.core.bridge.pydantic import PrivateAttr
# from llama_index.core.embeddings import BaseEmbedding

# class LlamaIndexEmbeddings(BaseEmbedding):
#     _model: emv.features.text.TextEmbedder = PrivateAttr()
#     def __init__(
#         self,
#         **kwargs: Any,
#     ) -> None:
#         self._model = emv.features.text.TextEmbedder()
#         super().__init__(**kwargs)

#     @classmethod
#     def class_name(cls) -> str:
#         return "LlamaIndexEmbeddings"

#     async def _aget_query_embedding(self, query: str) -> List[float]:
#         return self._get_query_embedding(query)

#     async def _aget_text_embedding(self, text: str) -> List[float]:
#         return self._get_text_embedding(text)

#     def _get_query_embedding(self, query: str) -> List[float]:
#         embeddings = self._model.encode(query)
#         return embeddings[0]

#     def _get_text_embedding(self, text: str) -> List[float]:
#         embeddings = self._model.encode(text)
#         return embeddings[0]

#     def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
#         embeddings = self._model.encode(texts)
#         return embeddings